In [3]:
%pip install imutils
%pip install keras
%pip install tensorflow

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import os
import pickle

from imutils import paths

import numpy as np
import tensorflow

from keras.models     import Sequential,Model
from keras.layers     import *
from keras.optimizers import *
from keras            import applications
from keras            import backend as K

import boto3

In [5]:
IMAGE_SHAPE                = (96, 96, 3)
CONVNET_DROPOUT            = 0.6
CONVNET_HIDDEN_LAYER1_SIZE = 4096
CONVNET_HIDDEN_LAYER2_SIZE = 4096

DEEPRANK_KERNEL_SIZE       = (8, 8)
DEEPRANK_CONV_STRIDES1     = (16, 16)
DEEPRANK_CONV_STRIDES2     = (32, 32)

S3_BUCKET_NAME        = "cs230project"
TRIPLET_WEIGHTS_FNAME = "triplet_weight.hdf5"
DATASET_FOLDER_NAME   = "dataset"

In [6]:
def download_s3_folder(s3_bucket, s3_folder, local_dir=None):
    for obj in s3_bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        s3_bucket.download_file(obj.key, target)

In [7]:
def build_convnet_model():
    vgg_model     = applications.VGG16(weights=None, include_top=False, input_shape=IMAGE_SHAPE)
    x             = vgg_model.output
    x             = GlobalAveragePooling2D()(x)
    x             = Dense(CONVNET_HIDDEN_LAYER1_SIZE, activation='relu')(x)
    x             = Dropout(CONVNET_DROPOUT)(x)
    x             = Dense(CONVNET_HIDDEN_LAYER2_SIZE, activation='relu')(x)
    x             = Dropout(CONVNET_DROPOUT)(x)
    x             = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

In [8]:
def deep_rank_model():
    convnet_model = build_convnet_model()

    first_input = Input(shape=IMAGE_SHAPE)
    first_conv  = Conv2D(96, kernel_size=DEEPRANK_KERNEL_SIZE, strides=DEEPRANK_CONV_STRIDES1, padding='same')(first_input)
    first_max   = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max   = Flatten()(first_max)
    first_max   = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=IMAGE_SHAPE)
    second_conv  = Conv2D(96, kernel_size=DEEPRANK_KERNEL_SIZE, strides=DEEPRANK_CONV_STRIDES2, padding='same')(second_input)
    second_max   = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max   = Flatten()(second_max)
    second_max   = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])

    emb = Dense(4096)(merge_two)
    emb = Dense(100)(emb)

    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model

In [9]:
def generate_encodings():
    imagePaths = list(paths.list_images(DATASET_FOLDER_NAME))

    knownEncodings = []
    knownNames     = []
    knownIDs       = []

    for (i, imagePath) in enumerate(imagePaths):
        info  = imagePath.split(os.path.sep)[-2]
        print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
        info = (info.split('_'))
        name = info[0]
        id = info[1]
        print(info)
        face = cv2.imread(imagePath)
        face = cv2.resize(face,(96,96))
        face = face/255

        face=np.expand_dims(face,axis=0)
        encodings = model.predict([face,face,face])
        if(len(encodings) != 1):
            print("error image")
            continue
        for encoding in encodings:
            print("Recognition Ok !")
            knownEncodings.append(encoding)
            knownNames.append(name)
            knownIDs.append(id)
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames, "ids": knownIDs}
    print(data)
    f = open("encodings.pickle", "wb")
    f.write(pickle.dumps(data))
    f.close()

In [10]:
model = deep_rank_model()
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 96, 96, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 96, 96, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 48, 48, 64)   0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [11]:
s3        = boto3.resource('s3')
s3_bucket = s3.Bucket(S3_BUCKET_NAME)

In [12]:
s3_bucket.download_file(TRIPLET_WEIGHTS_FNAME, TRIPLET_WEIGHTS_FNAME)

In [13]:
model.load_weights(TRIPLET_WEIGHTS_FNAME)

In [14]:
download_s3_folder(s3_bucket, DATASET_FOLDER_NAME)

In [15]:
generate_encodings()

[INFO] processing image 1/35
['chung', '33']
Recognition Ok !
[INFO] processing image 2/35
['thao', '01']
Recognition Ok !
[INFO] processing image 3/35
['nam', '02']
Recognition Ok !
[INFO] processing image 4/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 5/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 6/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 7/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 8/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 9/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 10/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 11/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 12/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 13/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 14/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 15/35
['quoc', '0020']
Recognition Ok !
[INFO] processing image 16/35
['quoc', '0020']
Recognit